\section{Hamiltonian}

\begin{align}
    H &= H_K + H_{h} \\
    H_K &= - \sum_{<j,k>_\alpha} J_\alpha \sigma_j^\alpha \sigma_k^\alpha \\
    H_h &= - \sum_\alpha h_\alpha \sigma_0^\alpha
\end{align}

The ground state Hamiltonain of Kitaev model is :
\begin{equation}
    H_K^0 = \sum_{<j,k>_\alpha} J_\alpha \ i c_j c_k
\end{equation}

When applied the local magnetic field, the $<u_{<0j>_\alpha}$ is not conserved anymore while others are still conserved quanties. We can rewrite the Hamiltonian into this form:

\begin{align}
\begin{split}
    H &= H_K^0 + \sum_{<0j>_\alpha} J_\alpha \ (i b_0^\alpha b_j^\alpha - 1) \ i c_0 c_j
            - \sum_\alpha h_\alpha \ i c_0 b_0^\alpha \\
      & = H_K^0 + H_{int} + H_h\\
      & = H_K^0 + H_{imp}
\end{split}
\end{align}

where $H_{imp} = H_{int} + H_h$.


\section{MFT Hamiltonian}

Apply MFT to $H_{int}$, we have:
\begin{equation}
    H_{int}^{MF} = \sum_\alpha J_\alpha ( <i c_0 c_\alpha> i b_0^\alpha b_\alpha^\alpha  
                        + <i b_0^\alpha b_\alpha^\alpha> i c_0 c_\alpha
                        - <i c_0 b_0^\alpha> i c_\alpha b_\alpha^\alpha
                        - <i c_\alpha b_\alpha^\alpha> i c_0 b_0^\alpha)
\end{equation}
The constant term is:
\begin{equation}
    H_{int}^{C} = \sum_\alpha J_\alpha( <i c_0 b_0^\alpha> <i c_\alpha b_\alpha^\alpha>
                                    - <i c_0 c_\alpha><i b_0^\alpha b_\alpha^\alpha>)
\end{equation}

where we use superscript $\alpha$ to label bonds direction and subscript $\alpha$ to label atoms link to the $0$-atom with bond $\alpha$. And we ignore the $(i c_0 b_\alpha^\alpha)(i b_0^\alpha c_\alpha)$ terms because they have been proved $0$ in previous studies.

\section{Lattice}

We use a $N_1 \times N_2 \times 2$ rhomboid lattice. The lattice vectors are $\vec{n}_1 = (0,1), \vec{n}_2 = (\frac{1}{2},\frac{\sqrt{3}}{2})$. The number of unit cells along these two directions are $N_1$ and $N_2$ respectively. We label each unit cell by its row and colume numbers $(r, c)$(start from $0$), the label of this unit cell given by $ n = r \times N_1 + c$. We can label A atom in the $n$-th unit cell as $2n$ and B atom as $2n + 1$.

For a given A atom $2n$, the B atom linked to it by $x$-bond is $2n + 1$, $y$-bond is $2n - 1$, $z$-bond is $2 (n - N_1) + 1$.

we use the basis $(c_{1A},\ c_{1B},\ ... \,\ c_{NA},\ c_{NB})^T$ when there is no magnetic field. When $H_h$ is added, the basis dimension becomes $N_1 \times N_2 \times 2 + 6$,  where the last six operators in the basis is $(b_0^x,\ b_0^y,\ b_0^z,\ b_x^x,\ b_y^y,\ b_z^z)$.

In [54]:
import numpy as np
from scipy.sparse import csc_matrix

def BulkHam(JK, N1,N2, periodic = False, flux = False):
    '''Kitaev Hamiltonian in fixed flux pattern
    
       Inputs: JK : 1*3 matrix, Jx=J[0], Jy = J[1], Jz=J[2]
               N1,N2: int
               
       Return: H matrix'''
    
    SiteNum = N1 * N2 * 2
    Hr = np.zeros([SiteNum, SiteNum],dtype=complex)
    for r in range(N2):
        for c in range(N1):
            n = r * N1 + c #label of unit cell
            '''X-bonds'''
            next = 2 * n + 1
            Hr[2*n, next] +=  1j * JK[0]  
                
            '''Y-bonds'''
            if c > 0:
                next = 2*n -1
            elif c == 0 and periodic == True:
                next = 2 * (n + N1) -1
            else:
                next = 0.5
                    
            if next % 1 == 0:
                Hr[2*n, next] += 1j * JK[1]
   
            '''Z-bonds'''
            if r > 0:
                next = 2*(n - N1) + 1
            elif r == 0 and periodic == True:
                next = 2 * (n + N1 * N2 - N1) +1
            else:
                next = 0.5
                
            if next % 1 == 0:
                if flux == True and r == N2//2 and c >= N1//2:
                    Hr[2*n, next] +=  -1j * JK[2]
                else:
                    Hr[2*n, next] +=  1j * JK[2]
                    
    Hr = (Hr + np.conj(Hr).T)/2
    return Hr

def BulkHam_csc(JK, N1,N2, periodic = False, flux = False):
    '''Kitaev Hamiltonian in fixed flux pattern
    
       Inputs: JK : 1*3 matrix, Jx=J[0], Jy = J[1], Jz=J[2]
               N1,N2: int
               
       Return: '''
    
    SiteNum = N1 * N2 * 2
    
    row = []
    col = []
    data = []
    for r in range(N2):
        temp = {} # A temporary dict
        for c in range(N1):
            n = r * N1 + c #label of unit cell
            '''X-bonds'''
            next = 2*n + 1
            if next in temp:
                temp[next] += 1j * JK[0]
            else: 
                temp[next] = 1j * JK[0]
                
            '''Y-bonds'''
            if c > 0:
                next = 2*n -1
            elif c == 0 and periodic == True:
                next = 2 * (n + N1) -1
            else:
                next = 0.5
                    
            if next % 1 == 0:
                if next in temp:
                    temp[next] += 1j * JK[1]
                else: 
                    temp[next] = 1j * JK[1]
   
            '''Z-bonds'''
            if r > 0:
                next = 2*(n - N1) + 1
            elif r == 0 and periodic == True:
                next = 2 * (n + N1 * N2 - N1) +1
            else:
                next = 0.5
                
            if next % 1 == 0:
                if next in temp:
                    if flux == True and r == N2//2 and c >= N1//2:
                        temp[next] +=  -1j * JK[2]
                    else:
                        temp[next] +=  1j * JK[2]
                else: 
                    if flux == True and r == N2//2 and c >= N1//2:
                        temp[next] =  -1j * JK[2]
                    else:
                        temp[next] =  1j * JK[2]
            odTemp = sorted(temp)
            for e in odTemp:
                row.append(2*n)
                col.append(e)
                data.append(temp[e])
                    
    return row, col, data

def ImpHam_csc(h, parameters):
    '''H_imp = H_int + H_h
    
       Inputs: h:1*3 matrix, hx=h[0], hy = h[1], hz=h[2]
               parameters: dict{order parameter: value}
               '''
    return 

def makeH(row, col, data, dim):
    '''Make csc format Hamiltonian
       
       Inputs : row,col :position of the non-zero elementds of Hamiltonian matrix
                data: H(row,col) = data
                dim: Hamiltonian matrix dimension
       Return: Hcsc: csc formate Hamiltonian
       
       Notice: we only have to enter half of the elements and get the other half 
               by hermition conjugate.'''
    Hcsc = csc_matrix((data, (row, col)), shape=(dim, dim),dtype = complex)
    Hcsc = (Hcsc + Hcsc.getH()) /2 
    return Hcsc
    

In [53]:
JK = np.array([10,1,0.1])
N1 = 2
N2 = 3
SiteNum = N1*N2 *2
H0 = BulkHam(JK, N1,N2, periodic = True, flux = True)
H1 = BulkHam_csc(JK, N1,N2, periodic = True, flux = True)
print(H1.count_nonzero()/ (SiteNum * SiteNum))

0.2916666666666667


In [11]:
import numpy as np
from scipy.sparse import csc_matrix

row = np.array([0, 2, 2, 0, 1, 2])
col = np.array([0, 0, 1, 2, 2, 2])
data = np.array([1, 2, 3, 4, 5, 6])
A = csc_matrix((data, (row, col)), shape=(3, 3))
B = A+A
print(A.toarray())
print((A+A).toarray())

[[1 0 4]
 [0 0 5]
 [2 3 6]]
[[ 2  0  8]
 [ 0  0 10]
 [ 4  6 12]]


In [13]:
A.getH().toarray()

array([[1, 0, 2],
       [0, 0, 3],
       [4, 5, 6]], dtype=int64)